In [1]:
# %%
# built in
%matplotlib inline
%load_ext autoreload
%autoreload 0
%reload_ext autoreload

import datetime as dt
import os
import pathlib
import sys
import warnings

from functools import partial
from importlib import reload

warnings.filterwarnings('ignore')

# third party
import ipywidgets as w
import matplotlib as mpl
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

from matplotlib import pyplot as plt
from scipy import stats
from statsmodels import api as sm
from tensorflow import keras


scripts_path = pathlib.Path(os.getcwd()).parents[0]
if not str(scripts_path) in sys.path:
    sys.path.append(str(scripts_path))

# internal
import config
from features import app, transformers, selector

sns.set(style='whitegrid')
%matplotlib inline

Could not import r-package RCIT


### Data Loading and Pre Processing
1. The FeatureSelector constructor expects two inputs, namely the features dataframe and the target series.
2. Both features and target must be aligned in terms of DateTimeIndex.
3. It's also advisable that in the features dataframe the column names are unique (even though the selector class has a built in logic to handle the most common exceptions).
4. Finally all the Rankers used for feature selection assume some form of stationarity in the data.

The cell below should be used to preprocess the data in order to satisfy all the above mentioned criteria.

In [2]:
data = pd.read_csv(config.data_path / 'data_20191219_0910.csv', index_col=0, parse_dates=True, dayfirst=True)

offset = pd.offsets.Week()
target_col = 'FX_AUD'
n_features = 10

feature_cols = data.columns
dataw = data.resample(rule=offset).last().iloc[500:, :].loc[:, feature_cols]

X = dataw
y = dataw[target_col]
X = dataw.sample(n=n_features, axis=1)
X = pd.concat([X, y], axis=1)

transformer = transformers.DiffTransformer()
X = transformer.fit_transform(X)
X, y = X.drop(target_col, axis=1), X[target_col]

After transformation 0 non stationary features remaining out of 11 features []
0 failed features out of 11 features
the following columns will be dropped []


### Interactive application forFeature Selection
The api is pretty simple to use. <br>

1. Instanciate the MultiTab object with two arguments: feature dataframe and target series.
2. Call the app property to display the interactive application in a notebook cell.



In [3]:
reload(app)
reload(selector)

<module 'features.selector' from 'C:\\dev\\projects\\feature-selection\\features\\selector.py'>

In [4]:
fsapp = app.MultiTab(X, y)
fsapp.app

In [ ]:
self = fsapp.selector

In [ ]:
self.kbest(3)

In [ ]:
k = 10
to_exclude = []

# filtering features
methods_to_include_mask = ~ self.scores_agg_.columns.isin(to_exclude)

# creating boolean list for masking
# I cannot use apply because the mask depends both on ranks and scores (two different dataframes)
rankers_dict = {ranker.name: ranker for ranker in self.rankers_}
ss_list = []
for col in self.scores_agg_.loc[:, methods_to_include_mask]:
    ascending = not rankers_dict[col].ascending
    close_to_zero_mask = np.isclose(self.scores_agg_[col], 0) * ascending
    ss_kbest_bool = (self.ranks_agg_[col] <= k) & (~close_to_zero_mask)
    ss_list.append(ss_kbest_bool)

df_kbest_bool = pd.concat(ss_list, axis=1)
ss_kbest_ = (df_kbest_bool.sum(axis=1).sort_values(ascending=False) / df_kbest_bool.shape[1]).rename('kbest')

ss_kbest_ranks_ = ss_kbest_.rank(method='dense', ascending=False)
group = self.ranks_agg_.groupby(ss_kbest_ranks_)
df_list = []
for index, df in group:
    sorted_index = df.mean(axis=1).sort_index().index
    df_list.append(df.loc[sorted_index, :])
ss_kbest_ = pd.concat(df_list)

In [ ]:
for index, df in group:
    sorted_index = df.mean(axis=1).sort_index().index
    break

In [ ]:
ss_kbest_ = pd.concat(df_list)

In [ ]:
ss_kbest_.squeeze().rename('kbest')

In [ ]:
k = 4
to_exclude=None

if to_exclude is None:
    to_exclude = []

# filtering features
methods_to_include_mask = ~ self.scores_agg_.columns.isin(to_exclude)

In [ ]:
rankers_dict = {ranker.name: ranker for ranker in self.rankers_}

In [ ]:
ss_list = []
for col in self.scores_agg_.loc[:, methods_to_include_mask]:
    ascending = not rankers_dict[col].ascending
    close_to_zero_mask = np.isclose(self.scores_agg_[col], 0) * ascending
    ss_kbest_bool = (self.ranks_agg_[col] <= k) & (~close_to_zero_mask)
    ss_list.append(ss_kbest_bool)

In [ ]:
df_kbest_bool = pd.concat(ss_list, axis=1)
ss_kbest_ = (df_kbest_bool.sum(axis=1).sort_values(ascending=False) / df_kbest_bool.shape[1])

In [ ]:
ss_kbest_ranks_ = ss_kbest_.rank(method='dense', ascending=False)

In [ ]:
ss_kbest_ranks_

In [ ]:
df

In [ ]:
df.mean(axis=1).sort_values()

In [ ]:
group = self.ranks_agg_.groupby(ss_kbest_ranks_)
df_list = []
for index, df in group:
    mean_df = df.mean(axis=1).sort_values()
    rank_values = index + (0.99 / np.arange(mean_df.shape[0], 0, -1))
    mean_df[:] = rank_values
    df_list.append(mean_df)

In [ ]:
k = 4
to_exclude=None

if to_exclude is None:
    to_exclude = []

# filtering features
methods_to_include_mask = ~ self.scores_agg_.columns.isin(to_exclude)

# creating boolean list for masking
# I cannot use apply because the mask depends both on ranks and scores (two different dataframes)
rankers_dict = {ranker.name: ranker for ranker in self.rankers_}
ss_list = []
for col in self.scores_agg_.loc[:, methods_to_include_mask]:
    ascending = not rankers_dict[col].ascending
    close_to_zero_mask = np.isclose(self.scores_agg_[col], 0) * ascending
    ss_kbest_bool = (self.ranks_agg_[col] <= k) & (~close_to_zero_mask)
    ss_list.append(ss_kbest_bool)

df_kbest_bool = pd.concat(ss_list, axis=1)
ss_kbest_ = (df_kbest_bool.sum(axis=1).sort_values(ascending=False) / df_kbest_bool.shape[1])

ss_kbest_ranks_ = ss_kbest_.rank(method='dense', ascending=False)
group = self.ranks_agg_.groupby(ss_kbest_ranks_)
df_list = []
for index, df in group:
    mean_df = df.mean(axis=1).sort_values()
    rank_values = index + (0.99 / np.arange(mean_df.shape[0], 0, -1))
    sorted_index = df.mean(axis=1).sort_values().index
    df_list.append(mean_df)
ss_kbest_ = pd.concat(df_list).squeeze().rename('kbest')

In [ ]:
ss_kbest_